In [1]:
!pip install gpflow
!pip install rdkit
import gpflow
from gpflow.mean_functions import Constant
from gpflow.utilities import positive, print_summary
from gpflow.utilities.ops import broadcasting_elementwise
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from rdkit.Chem import AllChem, Descriptors, MolFromSmiles
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
class Tanimoto(gpflow.kernels.Kernel):
    def __init__(self):
        super().__init__()
        # We constrain the value of the kernel variance to be positive when it's being optimised
        self.variance = gpflow.Parameter(1.0, transform=positive())

    def K(self, X, X2=None):
        """
        Compute the Tanimoto kernel matrix σ² * ((<x, y>) / (||x||^2 + ||y||^2 - <x, y>))

        :param X: N x D array
        :param X2: M x D array. If None, compute the N x N kernel matrix for X.
        :return: The kernel matrix of dimension N x M
        """
        if X2 is None:
            X2 = X

        Xs = tf.reduce_sum(tf.square(X), axis=-1)  # Squared L2-norm of X
        X2s = tf.reduce_sum(tf.square(X2), axis=-1)  # Squared L2-norm of X2
        outer_product = tf.tensordot(X, X2, [[-1], [-1]])  # outer product of the matrices X and X2

        # Analogue of denominator in Tanimoto formula

        denominator = -outer_product + broadcasting_elementwise(tf.add, Xs, X2s)

        return self.variance * outer_product/denominator

    def K_diag(self, X):
        """
        Compute the diagonal of the N x N kernel matrix of X
        :param X: N x D array
        :return: N x 1 array
        """
        return tf.fill(tf.shape(X)[:-1], tf.squeeze(self.variance))


In [5]:

data = pd.read_csv('tg_raw.csv')
smiles_list = data['SMILES'].to_list()
property_vals = data['Tg'].to_numpy()
y = property_vals

In [6]:
rdkit_mols = [MolFromSmiles(smiles) for smiles in smiles_list]
X = [AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=2048) for mol in rdkit_mols]
X = np.asarray(X)
X.shape

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator
[07:50:50] DEPRECATION WARNING: please use MorganGenerator

(7174, 2048)

In [7]:
X.shape, y

((7174, 2048), array([219. , 270. , 248.9, ..., 371. , 391. , 400. ]))

# Gaussian Process Regression with a Tanimoto Kernel

In [8]:
# We define the Gaussian Process Regression Model using the Tanimoto kernel

m = None

def objective_closure():
    return -m.log_marginal_likelihood()

In [9]:
test_set_size = 0.2

In [10]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_set_size, random_state=42)

y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

k = Tanimoto()
m = gpflow.models.GPR(data=(X_train, y_train), mean_function=Constant(np.mean(y_train)), kernel=k, noise_variance=1)

# Optimise the kernel variance and noise level by the marginal likelihood

opt = gpflow.optimizers.Scipy()
opt.minimize(objective_closure, m.trainable_variables, options=dict(maxiter=100))
print_summary(m)

# mean and variance GP prediction

y_pred, y_var = m.predict_f(X_test)
# y_pred
y_pred = np.array(y_pred)
score = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print("\nTest R^2: {:.3f}".format(score))
print("Test RMSE: {:.3f} nm".format(rmse))
print("Test MAE: {:.3f} nm".format(mae))

╒═════════════════════════╤═══════════╤══════════════════╤═════════╤═════════════╤═════════╤═════════╤══════════╕
│ name                    │ class     │ transform        │ prior   │ trainable   │ shape   │ dtype   │    value │
╞═════════════════════════╪═══════════╪══════════════════╪═════════╪═════════════╪═════════╪═════════╪══════════╡
│ GPR.mean_function.c     │ Parameter │ Identity         │         │ True        │ ()      │ float64 │  400.033 │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼──────────┤
│ GPR.kernel.variance     │ Parameter │ Softplus         │         │ True        │ ()      │ float64 │ 3935.53  │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼──────────┤
│ GPR.likelihood.variance │ Parameter │ Softplus + Shift │         │ True        │ ()      │ float64 │  253.373 │
╘═════════════════════════╧═══════════╧══════════════════╧═════════╧═════════════╧══════

In [11]:
# Output R^2, RMSE and MAE on the test set
y_pred = np.array(y_pred)
print(y)
score = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print("\nTest R^2: {:.3f}".format(score))
print("Test RMSE: {:.3f} nm".format(rmse))
print("Test MAE: {:.3f} nm".format(mae))

[219.  270.  248.9 ... 371.  391.  400. ]

Test R^2: 0.899
Test RMSE: 35.251 nm
Test MAE: 23.277 nm


In [12]:
# Output R^2, RMSE and MAE on the train set
y_pred_train, y_var = m.predict_f(X_train)
y_pred_train = np.array(y_pred_train)
score_train = r2_score(y_train, y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
mae_train = mean_absolute_error(y_train, y_pred_train)

print("\nTrain R^2: {:.3f}".format(score_train))
print("Train RMSE: {:.3f} nm".format(rmse_train))
print("Train MAE: {:.3f} nm".format(mae_train))


Train R^2: 0.994
Train RMSE: 8.585 nm
Train MAE: 5.335 nm


In [13]:
# Cross validation 5 fold
from sklearn.model_selection import KFold
# ============================================
def train_predict_gp_tanimoto(target_column, test_set_size=0.2):
    print(f"\n--- Training model for: {target_column} ---")

    df = data[['SMILES', target_column]].dropna()

    # Convert SMILES to fingerprint
    smiles_list = df['SMILES'].tolist()
    mols = [MolFromSmiles(smi) for smi in smiles_list]
    fps = [AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=2048) for mol in mols]
    X = np.array(fps).astype(np.float64)
    y = df[target_column].values.reshape(-1, 1).astype(np.float64)

    # ================================
    # 5-Fold Cross Validation
    # ================================
    kf = KFold(n_splits=5, shuffle=True, random_state=2)
    mae_scores, rmse_scores, r2_scores = [], [], []

    for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"\nFold {fold + 1}/5")

        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Chuyển kiểu dữ liệu chính xác
        X_train = X_train.astype(np.float64)
        X_test = X_test.astype(np.float64)
        y_train = y_train.astype(np.float64)
        y_test = y_test.astype(np.float64)

        # Khởi tạo mô hình Gaussian Process với kernel Tanimoto
        k = Tanimoto()
        m = gpflow.models.GPR(
            data=(X_train, y_train),
            mean_function=Constant(np.mean(y_train)),
            kernel=k,
            noise_variance=1.0,
        )

        # Hàm objective closure cho tối ưu
        def objective_closure():
            return -m.log_marginal_likelihood()

        # Tối ưu bằng Scipy
        opt = gpflow.optimizers.Scipy()
        opt.minimize(objective_closure, m.trainable_variables, options=dict(maxiter=100))

        # Dự đoán
        y_pred, y_var = m.predict_f(X_test)
        y_pred = y_pred.numpy().flatten()
        y_test = y_test.flatten()

        # Tính toán các chỉ số đánh giá
        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        r2 = r2_score(y_test, y_pred)

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        r2_scores.append(r2)

        print(f"Fold {fold + 1} - MAE: {mae:.3f}, RMSE: {rmse:.3f}, R²: {r2:.3f}")

    # ================================
    # Tổng hợp kết quả cross-validation
    # ================================
    print("\n===== 5-Fold Cross-Validation Results =====")
    print(f"Mean MAE : {np.mean(mae_scores):.3f} ± {np.std(mae_scores):.3f}")
    print(f"Mean RMSE: {np.mean(rmse_scores):.3f} ± {np.std(rmse_scores):.3f}")
    print(f"Mean R²  : {np.mean(r2_scores):.3f} ± {np.std(r2_scores):.3f}")

    # ================================
    # Huấn luyện lại mô hình trên toàn bộ dữ liệu
    # ================================
    print("\nTraining final model on full dataset...")
    k_final = Tanimoto()
    m_final = gpflow.models.GPR(
        data=(X, y),
        mean_function=Constant(np.mean(y)),
        kernel=k_final,
        noise_variance=1.0,
    )

    opt = gpflow.optimizers.Scipy()
    opt.minimize(lambda: -m_final.log_marginal_likelihood(), m_final.trainable_variables, options=dict(maxiter=100))

    print_summary(m_final)

    return m_final, {
        "MAE_mean": np.mean(mae_scores),
        "RMSE_mean": np.mean(rmse_scores),
        "R2_mean": np.mean(r2_scores),
        "MAE_std": np.std(mae_scores),
        "RMSE_std": np.std(rmse_scores),
        "R2_std": np.std(r2_scores),
    }

# ============================================
# Train cho cột Tg
# ============================================
models = {}
for target in ['Tg']:
    models[target] = train_predict_gp_tanimoto(target)



--- Training model for: Tg ---
Số lượng điểm dữ liệu trong khoảng 50–320K: 7174


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator
[07:55:30] DEPRECATION WARNING: please use MorganGenerator


Fold 1/5
Fold 1 - MAE: 23.176, RMSE: 33.891, R²: 0.909

Fold 2/5
Fold 2 - MAE: 23.983, RMSE: 37.077, R²: 0.889

Fold 3/5
Fold 3 - MAE: 22.413, RMSE: 33.315, R²: 0.911

Fold 4/5
Fold 4 - MAE: 23.583, RMSE: 35.545, R²: 0.902

Fold 5/5
Fold 5 - MAE: 23.967, RMSE: 37.627, R²: 0.888

===== 5-Fold Cross-Validation Results =====
Mean MAE : 23.424 ± 0.586
Mean RMSE: 35.491 ± 1.696
Mean R²  : 0.900 ± 0.010

Training final model on full dataset...
╒═════════════════════════╤═══════════╤══════════════════╤═════════╤═════════════╤═════════╤═════════╤══════════╕
│ name                    │ class     │ transform        │ prior   │ trainable   │ shape   │ dtype   │    value │
╞═════════════════════════╪═══════════╪══════════════════╪═════════╪═════════════╪═════════╪═════════╪══════════╡
│ GPR.mean_function.c     │ Parameter │ Identity         │         │ True        │ ()      │ float64 │  400.067 │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────

## Predicted Tg: PPG, PI, PPS, PB
#### poly(propylene glycol)	CC(O)COC(C)CO
#### poly(isoprene)	        CC(C-*)=CC-*
#### poly(propylene sulfide)	CC1CS1
#### poly(butadiene)	        C=CC=C

In [14]:
TNT = ["CC(O)COC(C)CO",
        "CC(C-*)=CC-*",
        "CC1CS1",
        "C=CC=C"]
TNT = pd.Series(TNT)

rdkit_mols = [MolFromSmiles(smiles) for smiles in TNT]
TNT = [AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=2048) for mol in rdkit_mols]
TNT = np.asarray(TNT)

TNT = TNT.astype(np.float64)

y_TNT, _ = m.predict_f(TNT)
y_TNT

[08:24:45] DEPRECATION WARNING: please use MorganGenerator
[08:24:45] DEPRECATION WARNING: please use MorganGenerator
[08:24:45] DEPRECATION WARNING: please use MorganGenerator
[08:24:45] DEPRECATION WARNING: please use MorganGenerator


<tf.Tensor: shape=(4, 1), dtype=float64, numpy=
array([[321.26114803],
       [273.02045108],
       [404.30698517],
       [342.37424267]])>